In [1]:
#import the needed libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') #Supress unnecessary warnings for readability and cleaner presentation
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from IPython.display import display
import pickle
color = sns.color_palette()

import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x) #Limiting floats output to 3 decimal points
import gc
print (gc.isenabled())

True


In [2]:
#deeplearning
import tensorflow
import keras
from keras.models import Model,Sequential

from keras.layers.core import Dense, Reshape, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Embedding, merge
from keras import backend as K

Using TensorFlow backend.


In [3]:
# concatenate all product-ids into a single string
# thanks to https://www.kaggle.com/eoakley/start-here-simple-submission

def products_concat(series):
    out = ''
    for product in series:
           out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [4]:
def df_split(df):
    df_reordered = df[df['reordered'] == 1]
    df_reordered = df_reordered.groupby('order_id')['product_id'].apply(products_concat)
    
    try:
        df_reordered = df_reordered.reset_index()
        df_reordered.columns = ['order_id','products_list']
    except:
        df_reordered = df_reordered.reset_index(level = ['order_id'])
        df_reordered.columns = ['order_id','products_list']
        
    return df_reordered

In [5]:
def multilabel_fscore(ytrue, ypred):
    """
    ex1:
    y_true = [1, 2, 3]
    y_pred = [2, 3]
    return: 0.8
    
    ex2:
    y_true = ["None"]
    y_pred = [2, "None"]
    return: 0.666
    
    ex3:
    y_true = [4, 5, 6, 7]
    y_pred = [2, 4, 8, 9]
    return: 0.25
    
    """
    #y_true = K.eval(ytrue)
    #y_pred = K.eval(ypred)
    F1_score = []
    y_pred = []
    y_true = []
    for items in range(len(ytrue)):
        y_true = ytrue[items]
        y_pred = ypred[items]
        correct = sum([1 for i in y_pred if i in y_true])
        if correct > 0:

            precision = correct / len(y_pred)

            recall =    correct / len(y_true)

            F1_score.append((2 * precision * recall) / (precision + recall))

        else:
            F1_score.append(0)

    return np.mean(F1_score)

#if __name__ != '__main__':

print(multilabel_fscore([[2,3],['None']], [[2,3,4],['None']]))

0.9


In [ ]:
#Now let's get and put the data in  pandas dataframe
order_products_train = pd.read_csv('/training/order_products__train.csv')
order_products_prior = pd.read_csv('/training/order_products__prior.csv')
orders = pd.read_csv('/training/orders.csv',engine ='c')
#aisles = pd.read_csv('input/aisles.csv')
#departments = pd.read_csv('input/departments.csv')
display(order_products_prior.head(1))
display(orders.head(1))

In [ ]:
products = pd.read_csv('/training/products.csv')
value = list(products['product_id'].astype(str))
value.append('None')
#save data
pickle.dump(value, open('prod_dict.p', 'wb'), protocol=4)

In [ ]:
#create one hot encode of aisle and dept
df_additional = products[['product_id','aisle_id','department_id']]
df_additional[['aisle_id','department_id']] = df_additional[['aisle_id','department_id']].astype(str)
df_additional = pd.get_dummies(df_additional, columns=['aisle_id','department_id'])
display(df_additional.head(2))

In [ ]:
prior_reo = order_products_prior[order_products_prior['reordered'] == 1].drop(['reordered','add_to_cart_order'], axis=1)
display(prior_reo.head(1))
train_reo = order_products_train[order_products_train['reordered'] == 1].drop(['reordered','add_to_cart_order'], axis=1)
prior_reo = prior_reo.merge(train_reo,on=['order_id','product_id'], how = 'outer')
display(prior_reo.head(1))
prior_reo = prior_reo.merge(df_additional, on='product_id', how='left')
print('joined')
prior_reo = prior_reo.drop(['product_id'], axis =1)
print('merging')
prior_reo = prior_reo.merge(orders[['user_id','order_number','order_id']],on='order_id', how='left')
prior_reo = prior_reo.sort_values(['user_id','order_number'])
print (prior_reo.info())
print('aggregating')
prior_reo = prior_reo.groupby(['user_id','order_number', 'order_id']).sum()
display(prior_reo.head(5))
print('shifting previous orders')
prior_reo= prior_reo.groupby(level=0).shift(1)
#apply(lambda x: pd.rolling_sum(x, window=2, min_periods=0)
prior_reo = prior_reo.reset_index()
display(prior_reo.head(5))
#print('saving')
#prior_reo.to_csv('input/prior_prods.csv', index=False)
#display(prior_reo.head(2))

In [ ]:
order_prod_list = order_products_train.merge(order_products_prior, how = 'outer')
display(order_prod_list.head(1))
df_prior_prod = df_split(order_prod_list)
display(df_prior_prod.head(1))
df_train = orders.merge(df_prior_prod, on='order_id', how='left')
df_train = df_train.merge(prior_reo, on=['order_id','user_id','order_number'], how='left')
df_train['products_list'] = df_train['products_list'].fillna('None')
df_train = df_train.fillna(0)
display(df_train.tail(2))

In [ ]:
df_eval = df_train[df_train['eval_set'] == 'train']
display(df_eval.head(2))

In [ ]:
del order_products_train,order_products_prior,orders,products,order_prod_list
del df_prior_prod, prior_reo, df_additional
gc.collect()

In [ ]:
df_eval = df_eval.drop(['eval_set','user_id'], axis =1)

df_test =  df_train[df_train['eval_set'] == 'test']
df_train = df_train[df_train['eval_set'] == 'prior']
df_train = df_train.drop(['eval_set','user_id'], axis =1)
display(df_test.head(1))

In [ ]:
#save data
pickle.dump(df_eval, open('eval_orders.p', 'wb'), protocol=4)

pickle.dump(df_train, open('train_orders.p', 'wb'), protocol=4)

pickle.dump(df_test, open('test_orders.p', 'wb'), protocol=4)

## Checkpoint
data can be loaded from here

In [6]:
'''
evals = open('eval_orders.p','rb')
df_eval = pickle.load(evals)
evals.close()

train = open('train_orders.p','rb')
df_train = pickle.load(train)
train.close()
'''
df_train = pd.read_pickle(open('/training/train_orders.p','rb'))
'''
test = open('test_orders.p','rb')
test_orders = pickle.load(test)
test.close()
'''

"\ntest = open('test_orders.p','rb')\ntest_orders = pickle.load(test)\ntest.close()\n"

In [7]:
X_train = df_train.drop(['products_list'], axis = 1)

### setting up keras

In [8]:
X_train = X_train.set_index('order_id')
display(X_train.head(1))

,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,...,department_id_2,department_id_20,department_id_21,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7,department_id_8,department_id_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,2,8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [9]:
X_train[['order_dow', 'order_hour_of_day']] = X_train[['order_dow', 'order_hour_of_day']].astype(str)
X_train = pd.get_dummies(X_train, columns=['order_dow', 'order_hour_of_day'])
X_train = X_train.astype('int8')
display(X_train.head(1))
X_train = X_train.fillna(-1)
display(X_train.head(1))
X_train = X_train.replace(0,-1)
display(X_train.head(1))

,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1


In [10]:
#vectorise the products list
#from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
value = pickle.load( open('/dict/prod_dict.p', 'rb'))
dicts = []
for i in value:
    dicts.append(tuple(i.split()))
print (dicts[49688])


vectorizer = MultiLabelBinarizer(sparse_output=True).fit(dicts)

#prod_dict = {value:key  for (value, key) in zip(value, key)}
#print (list(vectorizer.classes_))
#print (key[49688])
#print (value[49688])
#vectorizer = TfidfVectorizer(vocabulary = prod_dict)
#vectorizer = CountVectorizer(vocabulary = prod_dict)
train_target = df_train['products_list'].values

print(train_target[1])
y_train = []
for i in train_target:
    y_train.append(tuple(i.split()))
print (y_train[0])
    

y_train = vectorizer.transform(y_train)
print(y_train.dtype)

('None',)
196 12427 26088
('None',)
int64


In [11]:
#deleted df
del df_train
gc.collect()
#custom round layer
from keras.layers import Layer
class Round(Layer):
    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)

    def get_output(self, train=False):
        X = self.get_input(train)
        return K.round(X)

    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
print (vectorizer.classes_[49688])
y_train = y_train.astype('int8')
print(y_train.dtype)

None
int8


In [12]:
#from keras.utils
#y = y_train[:200000].todense()
#y = to_categorical(y, num_classes=None)
y_eval = y_train[199900:200000].todense()
#y = y[:199900]
#from scipy.sparse import csr_matrix
#test = csr_matrix(y)
#print(vectorizer.inverse_transform(test[1]))

In [ ]:
from sklearn.utils import class_weight

class_weight = class_weight.compute_class_weight('balanced', np.unique(vectorizer.classes_), y)

In [106]:
print(X_train.shape)
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
# create model
model = Sequential()
model.add(Dense(212,input_dim=188, activation='relu'))
# Hidden layers
model.add(BatchNormalization())
model.add(Dense(316,activation='relu'))
model.add(Dense(486,activation='relu'))
model.add(Dense(624,activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(700,activation='relu'))
model.add(Dense(900,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))
#output layer
model.add(Dense(49689, activation='sigmoid'))

#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.99, epsilon=1e-03, decay=0.0001)
RMSprop = keras.optimizers.RMSprop(lr=0.0001)
#Nadam = keras.optimizers.Nadam(lr=0.001)
#Adagrad = keras.optimizers.Adagrad(lr=0.001)
try:
    #load weights
    model.load_weights("weights.best.hdf5")
    print('loaded on weights')
except:
    print('load weights failed')
    model = load_model('my_model.hdf5')

model.compile(loss='binary_crossentropy',optimizer= RMSprop, metrics = ['accuracy'])

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
keras_train = np.array(X_train)
keras_eval = np.array(X_train[199900:200000])

print (y_eval.shape)
display(X_train.head(1))

(3214874, 188)
loaded on weights
(100, 49689)


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1


In [26]:
def batch_generator(X, y, batch_size):
    number_of_batches = X.shape[0]/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(X)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index]
    y =  y[shuffle_index, :]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch]
        y_batch = y[index_batch,:].todense()
        counter += 1
        yield(X_batch,np.array(y_batch))
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0
def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]


In [105]:
print('Training')

batch_size = 1000
epochs = 5
n_batches = 2

tot_training = min(batch_size*n_batches,keras_train.shape[0])
print('tot items to train: {}'.format(tot_training))
# Fit the model

#hist = model.fit_generator(generator=batch_generator(keras_train, y_train, 1000),
#                    nb_epoch=1, 
#                    samples_per_epoch=1000,verbose=2,validation_data = ( keras_eval, y_eval ))

    
# Training cycle
for epoch in range(epochs):
    # Loop over all batches
        print('starting epoch {}'.format(epoch))
        for batch_features, batch_labels in batch_features_labels(keras_train[:tot_training], y_train[:tot_training],batch_size):
            y = batch_labels.todense()
            model.fit(batch_features, y,batch_size=990,epochs=1,verbose=0, validation_split=0.001,callbacks= callbacks_list, shuffle='false')
        #pred_class(y_eval,keras_eval)

            
#model.fit(keras_train, y, epochs=10, batch_size=1000,verbose=2, validation_split = 0.01, class_weight=class_weight) 

#score = model.evaluate(keras_eval, y_eval)

#print('\nTest score:', score[0])
#print('Test accuracy:', score[1])



Training
tot items to train: 2000
starting epoch 0


ResourceExhaustedError: OOM when allocating tensor with shape[999,49689]
	 [[Node: logistic_loss_14/zeros_like = ZerosLike[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Log_18)]]
	 [[Node: Mean_82/_1981 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_32_Mean_82", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'logistic_loss_14/zeros_like', defined at:
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "

In [99]:
model.save("my_model.hdf5")

def pred_class(y_eval,keras_eval):
        from scipy.sparse import csr_matrix
        print('Predicting')
        predicted_output = model.predict(keras_eval)
        predicted_output[predicted_output>=0.5] = 1
        predicted_output[predicted_output<0.5] = 0
        pred = csr_matrix(predicted_output)
        y_eval_pred = vectorizer.inverse_transform(pred)
        print (multilabel_fscore(y_eval,y_eval_pred))

#print('true eval set values {}'.format(y_eval[0]))
#print('pred eval set values {}\n'.format(y_eval_pred[0]))
print (pred_class(y_eval,keras_eval))

Predicting
0.0
None
